In [35]:
import cv2 as cv
import numpy as np
import os
import random


def get_image_names_from_dir(path):
    files = os.listdir(path)
    files = [f"{path}/{file_name}" for file_name in files]
    return files


def gen_kps_and_des(image_list, maxKeyPoints=800):
    sift = cv.SIFT_create(maxKeyPoints)
    image_kp_des_list = []

    for image in image_list:
        kp, des = sift.detectAndCompute(image, None)
        image_kp_des_list.append([kp, des])

    return image_kp_des_list


def match_kps_knn(feat1, feat2, ratio=0.75):
    bf = cv.BFMatcher()
    rawMatches = bf.knnMatch(feat1, feat2, 2)
    matches = []

    for m, n in rawMatches:
        if m.distance < n.distance * ratio:
            matches.append(m)
    return matches


def gen_homography(kps1, kps2, matches, reproj_thres):
    # convert the keypoints to numpy arrays
    kps1 = np.float32([kp.pt for kp in kps1])
    kps2 = np.float32([kp.pt for kp in kps2])

    # construct the two sets of points
    pts1 = np.float32([kps1[m.queryIdx] for m in matches])
    pts2 = np.float32([kps2[m.trainIdx] for m in matches])

    # estimate the homography between the sets of points
    (homography, status) = cv.findHomography(pts1, pts2, cv.RANSAC, reproj_thres)

    return (homography, status)

image_names = get_image_names_from_dir("dataset/set1")
image_list = [cv.imread(image) for image in image_names]
image_gray_list = [cv.cvtColor(image, cv.COLOR_BGR2GRAY) for image in image_list]
image_idx_1 = 10
image_idx_2 = 11

img_1 = image_list[image_idx_1]
img_2 = image_list[image_idx_2]

img_gray_1 = image_gray_list[image_idx_1]
img_gray_2 = image_gray_list[image_idx_2]


In [36]:
# Computing keypoints, and descriptors using SIFT algorithm
image_kp_des_list = gen_kps_and_des(image_list)
kp_1, d10 = image_kp_des_list[image_idx_1]
kp_2, d11 = image_kp_des_list[image_idx_2]

# extract 200 random keypoint samples
random_sample_1 = random.sample(kp_1, 200)

img_sift_res_1 = cv.drawKeypoints(
    img_gray_1,
    random_sample_1,
    img_1,
    flags=cv.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS,
)

cv.imwrite("results/set1_200.jpg", img_sift_res_1)
cv.waitKey(0)
cv.destroyAllWindows()

In [38]:
# Matching feature descriptors between two images using Euclidean Distance
matches = match_kps_knn(d10, d11)
result_img = cv.drawMatches(
    img_gray_1,
    kp_1,
    img_gray_2,
    kp_2,
    matches,
    None,
    flags=2,
)

cv.imwrite("results/set1_matches.jpg", result_img)
cv.waitKey(0)
cv.destroyAllWindows()

In [ ]:
# Applying RANSAC to remove outliers
homography, mask = gen_homography(kp_1, kp_2, matches, 5)

# Filter out outliers
inlier_matches = [matches[i] for i in range(len(matches)) if mask[i] == 1]

# Draw matches with inliers
result_img = cv.drawMatches(
    img_gray_1,
    kp_1,
    img_gray_2,
    kp_2,
    inlier_matches,
    None,
    flags=2,
)

# Display the result
cv.imwrite("results/set1_bestMatches.jpg", result_img)
cv.waitKey(0)
cv.destroyAllWindows()

## custom image analysis

In [20]:
img_12 = cv.imread("dataset/x_foto1A.jpg")
img_13 = cv.imread("dataset/x_foto1B.jpg")

image_kp_des_list = gen_kps_and_des([img_12, img_13], 600)

kp_12, des_12 = image_kp_des_list[0]
kp_13, des_13 = image_kp_des_list[1]
matches = match_kps_knn(des_12, des_13)
homography, mask = gen_homography(kp_12, kp_13, matches, 5)

# Warp images
result = cv.warpPerspective(
    img_12, homography, (img_12.shape[1] + img_13.shape[1], img_13.shape[0])
)
result[0 : img_13.shape[0], 0 : img_13.shape[1]] = img_13

cv.imwrite("results/custom_image_result.jpg", result)
cv.waitKey(0)
cv.destroyAllWindows()